In [2]:
#0 - ore, 1 - clay , 2 - obsidian, 3 - geode
blueprints = []
for line in lines:
    values = [int(v) for v in re.findall("\d+", line)]
    blueprints.append([[values[1],0,0,0], [values[2],0,0,0], [values[3],values[4],0,0], [values[5],0,values[6],0]])
blueprints[:3]

[[[4, 0, 0, 0], [4, 0, 0, 0], [2, 7, 0, 0], [4, 0, 13, 0]],
 [[4, 0, 0, 0], [4, 0, 0, 0], [4, 15, 0, 0], [3, 0, 8, 0]],
 [[4, 0, 0, 0], [4, 0, 0, 0], [4, 18, 0, 0], [4, 0, 9, 0]]]

In [48]:
blueprint = blueprints[1]

maxOreCost = max([robotCost[0] for robotCost in blueprint])
maxClaiCost = max([robotCost[1] for robotCost in blueprint])
maxObsidianCost = max([robotCost[2] for robotCost in blueprint])

def decide(minute, stash, robots, history):

    #mine
    prevStash = stash    
    stash = [x + y for (x, y) in zip(robots, stash)]       

    #print(str(minute) + "Stash: " + str(stash) + " robots: " + str(robots))            
    
    #No point of having more robots, then material cost of next robot
    if robots[0] > blueprint[1][0] or robots[1] > blueprint[2][1] or robots[2] > blueprint[3][2]:
        return []

    if minute == 18:
        #keep only options with obsidian robot                
        if robots[2] == 0: 
            return []   

    if minute == 6:            
        if robots[0] + robots[1] < 2: 
            return []  
        
    if minute == 11:                
        if robots[0] + robots[1] < 4: 
            return []      
        
    if minute == 32:
        return [(robots, stash, history)]        

    options = []
    #if we have stash for every robot, build something!
    if prevStash[0] < maxOreCost or prevStash[1] < maxClaiCost or prevStash[2] < maxObsidianCost:
        if minute < 20 or prevStash[0] < maxOreCost or prevStash[2] < maxObsidianCost:
            options += (decide(minute+1, stash, robots, history))

    #build robots
    robotNumbers = [0,1,2,3]                 
    if minute >= 28: #build only geode robots after n minute
        robotNumbers = [3]
    elif minute >= 22:
        robotNumbers = [2,3]        
    for robotNumber in robotNumbers:        
        availableStash = [balance - cost for (balance, cost) in zip(prevStash, blueprint[robotNumber])]    
        if all([balance >= 0 for balance in availableStash]):
            #print("Building " + str(robotNumber) + " robot at minute " + str(minute))
            localStash = [balance - cost for (balance, cost) in zip(stash, blueprint[robotNumber])]
            localRobots = robots.copy()
            localRobots[robotNumber] += 1
            options += (decide(minute+1, localStash, localRobots, history + [robotNumber]))

    return options

#[(robots, stash)]
results = decide(1, [0,0,0,0], [1,0,0,0], [])        

print(len(results), end=", ")
maxGeodes = max([result[1][3] for result in results])
print(maxGeodes)

221568, 23


In [ ]:
#blueprint1: 29  [0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 3, 2, 3, 2, 3, 3, 3, 3]
#blueprint2: 23  [0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 3, 2, 3, 2, 3, 3, 3,3]
#blueprint3: 16  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 3, 2, 3, 2, 3, 3, 3]
29 * 23 * 16

In [44]:
#[[1,1,1,2,1,2,3,3]]    
blueprint = blueprints[1]
endStashStates = []
#order = [0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 3, 2, 3, 2, 3, 3, 3, 3] #blueprint 0
order = [0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 3, 2, 3, 2, 3, 3, 3,3] #blueprint 1
#order = [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 3, 2, 3, 2, 3, 3, 3] #blueprint2
robots = [1,0,0,0]
stash = [0, 0, 0, 0]
for curRound in range(32):            
    #mine                    
    prevStash = stash
    stash = [x + y for (x, y) in zip(robots, stash)]    

    #build robots
    if len(order) == 0:
        print("Minute " + str(curRound+1) + " robots: " + str(robots) +"prev stash: " +str(prevStash))                        
        continue

    robotToBuild = order[0]                     
    availableStash = [balance - cost for (balance, cost) in zip(prevStash, blueprint[robotToBuild])]
    if all([balance >= 0 for balance in availableStash]):
        stash = [balance - cost for (balance, cost) in zip(stash, blueprint[robotToBuild])]
        robots[robotToBuild] += 1
        order.remove(robotToBuild)
        print("Minute " + f"{curRound+1:02d}" + " Building robot " +str(robotToBuild) + " robots: " + str(robots) +" prev stash: " +str(prevStash))
    else:
        print("Minute " + f"{curRound+1:02d}" + "                  robots: " + str(robots) +" prev stash: " +str(prevStash))                

#if stash[3] > 8:
print("remaining order: " + str(order)+" stash: " + str(stash) + " robots: " + str(robots))        
endStashStates.append(stash)

maxGeodes = max([stash[3] for stash in endStashStates])
print("Max geodes: " + str(maxGeodes))

Minute 01                  robots: [1, 0, 0, 0] prev stash: [0, 0, 0, 0]
Minute 02                  robots: [1, 0, 0, 0] prev stash: [1, 0, 0, 0]
Minute 03                  robots: [1, 0, 0, 0] prev stash: [2, 0, 0, 0]
Minute 04                  robots: [1, 0, 0, 0] prev stash: [3, 0, 0, 0]
Minute 05 Building robot 0 robots: [2, 0, 0, 0] prev stash: [4, 0, 0, 0]
Minute 06                  robots: [2, 0, 0, 0] prev stash: [1, 0, 0, 0]
Minute 07                  robots: [2, 0, 0, 0] prev stash: [3, 0, 0, 0]
Minute 08 Building robot 0 robots: [3, 0, 0, 0] prev stash: [5, 0, 0, 0]
Minute 09                  robots: [3, 0, 0, 0] prev stash: [3, 0, 0, 0]
Minute 10 Building robot 1 robots: [3, 1, 0, 0] prev stash: [6, 0, 0, 0]
Minute 11 Building robot 0 robots: [4, 1, 0, 0] prev stash: [5, 0, 0, 0]
Minute 12 Building robot 1 robots: [4, 2, 0, 0] prev stash: [4, 1, 0, 0]
Minute 13 Building robot 1 robots: [4, 3, 0, 0] prev stash: [4, 2, 0, 0]
Minute 14 Building robot 1 robots: [4, 4, 0, 0] pre